In [ ]:
!pip install sentence_transformers
!pip install transformers

In [ ]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from nltk import sent_tokenize
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import nltk
import re
nltk.download('punkt')

## PROCESO NEURONAL INFORMATION RETRIEVAL

In [ ]:
class Corpus_esg:

    def __init__(self, empresa, año):
        self.empresa = empresa
        self.año = año
        self.passages=[]
    def get_path(self):
        """me da el path correspondiente de ese año esa empresa"""
        path_origen='/content/drive/MyDrive/DATO-ingles/'+self.empresa+'/'+str(self.año)+'/'
 
        return path_origen

    def get_archivos(self,formato):
          '''
          Coge los archivos de cada empresa y año que sean de un formato determinado
          Args:
                    formato (str): se cogeran los archivos de este formato , por defecto txt

          Return : una lsita con los archivos del formato indicado
          '''
        path_origen=self.get_path()
  
        contenido = os.listdir(path_origen)
        for archivo in contenido:
          tipo=archivo[-3:]
          if tipo !=formato:
            contenido.remove(archivo)
        
        return contenido

    def almacen_texto(self,formato='txt'):
      '''
      Divide el contenido de los archivos en parrafos
      Args:
              
                formato (str): se cogeran los archivos de este formato , por defecto txt

      Return : una lista con los parrafos de todos los archivos del formato indicado 
      '''
      contenido=self.get_archivos(formato=formato)
      passages = []
      for archivo in contenido:
        nombre=archivo[:-4]
        print(nombre)
        path=self.get_path()+nombre+'.'+formato
        document = open(path, 'r', encoding="utf-8")
        document=document.read() 
        paragraphs = []
        for paragraph in document.replace("\r\n", "\n").split("\n\n"):
              if len(paragraph.strip()) > 0:
                  paragraphs.append(sent_tokenize(paragraph.strip()))

        window_size = 20
        
        for paragraph in paragraphs:
              for start_idx in range(0, len(paragraph), window_size):
                  end_idx = min(start_idx+window_size, len(paragraph))
                  passages.append(" ".join(paragraph[start_idx:end_idx]))
      return passages
    def cargar_almacen_texto(self):
      '''
      Guardar los parrafos como argumento de la clase
      Return : una lista con los parrafos de todos los archivos del formato indicado 
      '''
      passages=self.almacen_texto()
      self.passages=passages

      return passages

    def descargar_almacen_texto(self):
      '''
      Guardar los parrafos en drive en formato csv

      
      '''
      dicc = {'passages': self.passages} 
      df = pd.DataFrame(dicc)
      nombre='corpus.csv'
      path=self.get_path()+nombre
      df.to_csv(path)
    def cargar_almacen_texto_drive(self):
      '''
      carga los parrafos que estan guardados en el drive

      Return : una lista con los parrafos 
      '''
      path=self.get_path()+'corpus.csv'
      datos=pd.read_csv(path)
      passages=datos['passages'].tolist()
      return passages

In [ ]:
class Retriever_esg:
      def __init__(self, corpus, pregunta):
        self.corpus = corpus
        self.pregunta = pregunta
        self.corpus_retriever=[]
      def passage_embedding(self):
        '''
        Realiza el embedding de los parrafos con el modelo que hemos elegido

        '''
        passage_encoder = SentenceTransformer('facebook-dpr-ctx_encoder-single-nq-base')

        passage_embeddings = passage_encoder.encode(self.corpus)
        return passage_embeddings

      def query_embedding(self):
        '''
        Realiza el embedding de la query con el modelo que hemos elegido

        '''
        query_encoder = SentenceTransformer('facebook-dpr-question_encoder-single-nq-base')
        query_embedding = query_encoder.encode(self.pregunta)
        return query_embedding

      def retriever(self):
        '''
        Realiza el document retriever: para ello realiza el embedding de los parrafos y la query, para 
        codificar la query y los passages en un espacio dimensinal y luego le aplica el producto escalar como
        funcion de similitud para comparar cada parrafo y la query. 

        Una vez obtenida la similitud entre la query y cada parrafo se eligen como posibles parrafos aquellos que tiene 
        un 20% o menos de diferencia con el parrafo con mas similitud.

        Return : lista con aqueelos parrafos con una cierta similitud con la query. y que por tanto es posible
        que la respuesta este en uno de ellos
        '''
        document_retriever=[]
        num_passages=len(self.corpus)
        passage_embeddings=self.passage_embedding()
        query_embedding=self.query_embedding()
        scores = util.dot_score(query_embedding, passage_embeddings)
        maxima_probabilidad=max(list(scores.numpy()[0]))
        minima_probabilidad=maxima_probabilidad-20
        print(maxima_probabilidad,minima_probabilidad)
        for i in range(0,num_passages):
          score=list(scores.numpy()[0])[i]
          if score>minima_probabilidad:
            document_retriever.append(passages[i])
        return document_retriever

      def cargar_corpus_retirever(self):
        
        corpus_r=self.retriever()
        self.corpus_retirever=corpus_r
        return corpus_r

      def generar_contexto(self):
        '''
        Esta funcion se encarga de generar el contexto donde puede estar la respuesta, y para ello 
        junta todos los parrafos que el document retriever considera posibles de que este la pregunta

        Return : el texto de juntar todos los parrafos
        '''
        contexto=''
        corpus_r=self.corpus_retriever
        for passage in corpus_r:
          contexto +=passage
          contexto +='\n'
        return contexto

In [ ]:
class Reader_esg:
  def __init__(self, contexto, pregunta):
        self.contexto = contexto
        self.pregunta = pregunta
        
  def document_reader(self):
    '''
    Realiza el document reader: para ello utiliza el modelo que hemos estudiado: roberta-bas-squad2


    Return : devuelva la respuesta dada por el modelo y el porcentaje con el que cree que lo ha 
    hecho bien
    '''
    model_name = "deepset/roberta-base-squad2"
    nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
    QA_input = {
        'question': self.pregunta,
        'context': self.contexto
          }
    res = nlp(QA_input)
    answer = res['answer']
    score=res['score']
    return [answer,score,res]

### FUNCIONES AUXILIARES

In [ ]:
def años_con_datos(empresa):
  '''
  busca para cada empresa de que años tiene disponible informacion
  Args:
            empresa (str): empresa de la que se busca 


  Return : una lista con los años donde la empresa tiene informacion
  '''
  path_origen='/content/drive/MyDrive/DATO-ingles/'+empresa+'/'
  contenido = os.listdir(path_origen)
  #me devuelve una lsita con los años
  return contenido

In [ ]:
def crear_corpus(empresa, año):
      '''
      Dada un año y una empresa crea el corpus 
      Args:
            empresa str: empresa a estudiar
            año str: año a estudiar

        Returns:
            corpus
      '''
  # 1º paso:crear la clase
  cor=Corpus_esg(empresa, año)
  # 2º paso:crear corpus y guardar passages
  print('crear corpus')
  corpus=cor.cargar_almacen_texto()
  # 3º paso: guardar en drive para tenerlo descargado
  cor.descargar_almacen_texto()
  

  return corpus

In [ ]:
def crear_retriver(corpus,pregunta):
  # 1º paso:crear la clase
  ret=Retriever_esg(corpus,pregunta)
  # 2º paso:crear retriver y guardar en la clase
  corpus_retirever=ret.cargar_corpus_retirever()
  return corpus_retriever

In [ ]:
def crear_reader(corpus,pregunta):
  # 1º paso:crear la clase
  reader=Reader_esg(corpus,pregunta)
  # 2º paso:crear reader y sacar respuesta
  respuesta=ret.document_reader()
  return respuesta

In [ ]:
def imprimir_respuestas(respuesta, variable,question, empresa, año,dimension):

  '''
  Guarda las respuesta dadas por el document reader en drive.

  Args:
            respuesta (str): respuesta dada por el Reader
            question (str): query con la que se define la variable 
            variable (str): nombre de la variable que buscamos
            empresa(str): empresa de la que estamos buscando
            año(str): año donde estamos buscando
            dimension(str): dimension a la que pertenece la variable (e,g,s)

  
  '''

  nombre='/content/drive/MyDrive/DATO-ingles/resultados_busquedas/'+empresa+'_'+año+'_resultados_'+dimension+'.txt'
  answer=respuesta[0]
  score=respuesta[1]
  res=respuesta[2]
  f = open(nombre, 'a')
  
  f.write('\n')
  f.write('---------------------------------------------')
  f.write('\n')
  f.write('la variable que estamos buscando es: '+ variable)
  f.write('\n')
  f.write('la query con la que buscamos : ' + question)
  f.write('\n')
  f.write('la respuesta que da el modelo es : ' +str(answer) )
  f.write('\n')
  f.write('la respuesta tiene una fiabilidad de: ' + str(score))
  f.write('\n')
  f.write('la respuesta completa es: ' + str(res))
  f.write('---------------------------------------------')
  f.write('\n')
  f.close()

In [ ]:
def proceso_pipeline(corpus,diccionario_preguntas,dimension, empresa,año):
  '''
  funcion que realiza todo el proceso NIR

  1º paso: hacer el document_retriever
  2º paso: hacer el document_Reader, calcula la respuesta de la variable y guarda los resultados en drive

  Args:
            corpus (str): texto que contiene la informacion de la empresa para ese año
            diccionario_preguntas (dic): diccionario con las variables y sus respectivas querys
            empresa(str): empresa de la que estamos buscando
            año(str): año donde estamos buscando
            dimension(str): dimension a la que pertenece el diccionario (e,g,s)


  '''
  
  for variable in diccionario_preguntas:
    pregunta=diccionario_preguntas[variable]
    corpus_retriever=crear_retriver(corpus,pregunta)
    respuesta=crear_reade(corpus_retriever,pregunta)
    imprimir_respuestas(respuesta,variable=variable,question=pregunta, empresa=empresa, año=año, dimension=dimension)

### PROCESO DADO UNA EMPRESA

Definimos la empresa a estudiar

In [ ]:
empresa='acciona'

In [ ]:
#diccionario con los indicadores que buscamos y sus preguntas
preguntas_e={'recognized_env_standard':'Is the environmental management system certified through an international standard such as ISO 14001, JIS Q 14001 or EMAS?',
                 'external_audit':'Is the environmental management system verified by a specialized external audit?',
                 'total_emissions_co2':'total amount of co2 emissions in tons',
                 '2030_commitment':'has a commitment on the 2030 GHG according to the EU policy to reduce greenhouse gases?',
               'energy_use':'Do you have a policy to manage the use of energy to achieve savings by using energy efficiently or renewable energy?',
                'manage_waste':'Do you have a waste management policy to avoid or reduce waste through a more efficient use of resources?',
                'biodiversity':'Commitment to conserve biodiversity and ecosystems',
                'climate_strategy':'has a Report on vulnerability to climate change to identify opportunities and risks associated with climate change through the recommendations of the Task Force on Climate Related Financial Disclosures TCFD?',
                'water_use':'Do you have a policy to manage use of water (efforts to manage water-related risks and opportunities)?',
                  'clean_tech':'implement the use of electric or hybrid vehicles (gas-electric)?', 
                 'green_buildings':'improvement of the environmental performance of your real estate assets with owned buildings that have the green certificate?',
'renewable_energy':'Has a policy to generate renewable energy and/or allow renewable energy development through grid expansion and "green energy" offerings?',
 'waste_use':'total amount of waste used or disposed in tons',
             'waste_recycled':'total amount of recycled waste in tons',
             'water_used':'total amount of water discharged or used in m3',
             'water_recycled':'total amount of water that is of recycled origin or that is recycled in m3',
             'w_renewable_energy':'number of watts of renewable energy',
             'circular_economy':'circular economy policy that involver reusing, repairing, renewing and recycling existing materials and products for as long as possible'}

In [ ]:
años=años_con_datos(empresa)
for año in años:
      #para cada año se crea el corpus, este es igual para todas las preguntas
      corpus=crear_corpus(empresa, año)
      proceso_pipeline(corpus,diccionario_preguntas=preguntas_e,dimension='e',empresa,año) 